# Sleep Disorder Classification - GPU-Accelerated Training

Trains 5 models (KNN, SVM, RF, ANN, CNN) with Optuna optimization.

**GPU Features**: Mixed precision (AMP), batch size 512, pin_memory, async CUDA transfers.

> **Important**: Go to **Runtime → Change runtime type → T4 GPU** before running!

In [ ]:
# 1. Verify GPU
import torch
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_mem = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f'GPU: {gpu_name} ({gpu_mem:.1f} GB)')
    print(f'CUDA version: {torch.version.cuda}')
else:
    print('WARNING: No GPU detected!')
    print('Go to Runtime -> Change runtime type -> T4 GPU')

In [ ]:
# 2. Install dependencies
!pip install -q optuna imbalanced-learn

In [ ]:
# 3. Mount Google Drive
from google.colab import drive
import os

drive.mount('/content/drive')
project_path = '/content/drive/MyDrive/Sleep_Disorder_Project'
os.makedirs(project_path, exist_ok=True)
print(f"Models will be saved to: {project_path}")

In [ ]:
# 4. Upload Dataset
from google.colab import files

print("Upload 'sleep_dataset.csv':")
uploaded = files.upload()
for fn in uploaded.keys():
    print(f'Uploaded "{fn}" ({len(uploaded[fn])} bytes)')
    if fn != 'sleep_dataset.csv':
        os.rename(fn, 'sleep_dataset.csv')

---
## Source Code Setup
**Option A**: Clone from GitHub (recommended)

In [ ]:
# Uncomment to clone from GitHub
# !git clone https://github.com/Pradeep-1803/buhbuybnu.git
# %cd buhbuybnu

**Option B**: Embedded code (run cells below if NOT cloning)

In [ ]:
os.makedirs('src', exist_ok=True)

In [ ]:
%%writefile src/data_loader.py
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
import warnings

warnings.filterwarnings('ignore')


def load_and_process_data(filepath):
    """Loads and preprocesses the sleep dataset. Returns data WITHOUT SMOTE."""
    print(f"Loading data from {filepath}...")
    df = pd.read_csv(filepath)
    
    if 'Person ID' in df.columns:
        df = df.drop(columns=['Person ID'])
    
    df['Sleep Disorder'] = df['Sleep Disorder'].fillna('None')
    print("Class distribution:")
    print(df['Sleep Disorder'].value_counts())

    if 'Blood Pressure' in df.columns:
        df[['Systolic_BP', 'Diastolic_BP']] = df['Blood Pressure'].str.split('/', expand=True).astype(float)
        df = df.drop(columns=['Blood Pressure'])
    
    if 'BMI Category' in df.columns:
        df['BMI Category'] = df['BMI Category'].replace({'Normal Weight': 'Normal'})

    target_col = 'Sleep Disorder'
    categorical_cols = ['Gender', 'Occupation', 'BMI Category']
    numeric_cols = [col for col in df.columns if col not in categorical_cols + [target_col]]
    
    numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])
    categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))])

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_cols),
            ('cat', categorical_transformer, categorical_cols)
        ]
    )
    
    X = df.drop(columns=[target_col])
    y = df[target_col]
    
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
    
    print("Preprocessing features...")
    X_processed = preprocessor.fit_transform(X)
    
    X_train, X_test, y_train, y_test = train_test_split(
        X_processed, y_encoded, test_size=0.3, random_state=42, stratify=y_encoded
    )
    
    print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")
    return X_train, X_test, y_train, y_test, label_encoder


def apply_smote(X, y):
    """Apply SMOTE to balance classes. Use only on training data."""
    smote = SMOTE(random_state=42)
    X_res, y_res = smote.fit_resample(X, y)
    return X_res, y_res

In [ ]:
%%writefile src/models.py
import torch
import torch.nn as nn
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from torch.utils.data import Dataset

class SleepDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

class ANN(nn.Module):
    def __init__(self, input_dim, hidden_layers=1, units_per_layer=24, dropout_rate=0.2, num_classes=3):
        super(ANN, self).__init__()
        layers = []
        layers.append(nn.Linear(input_dim, units_per_layer))
        layers.append(nn.BatchNorm1d(units_per_layer))
        layers.append(nn.ReLU())
        layers.append(nn.Dropout(dropout_rate))
        for _ in range(hidden_layers - 1):
            layers.append(nn.Linear(units_per_layer, units_per_layer))
            layers.append(nn.BatchNorm1d(units_per_layer))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout_rate))
        layers.append(nn.Linear(units_per_layer, num_classes))
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

class CNN(nn.Module):
    def __init__(self, input_dim, filters=32, kernel_size=2, dropout_rate=0.3, num_classes=3):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=filters, kernel_size=kernel_size)
        self.bn1 = nn.BatchNorm1d(filters)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_rate)
        self.pool = nn.MaxPool1d(kernel_size=2)
        conv_out_size = input_dim - kernel_size + 1
        pool_out_size = conv_out_size // 2
        if pool_out_size <= 0:
            self.pool = nn.Identity()
            pool_out_size = conv_out_size
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(filters * pool_out_size, num_classes)

    def forward(self, x):
        x = x.unsqueeze(1)
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.dropout(x)
        x = self.flatten(x)
        x = self.fc(x)
        return x

def get_sklearn_model(name, params):
    if name == 'KNN':
        return KNeighborsClassifier(**params, n_jobs=-1)
    elif name == 'SVM':
        return SVC(**params)
    elif name == 'RF':
        return RandomForestClassifier(**params, n_jobs=-1)
    else:
        raise ValueError(f"Unknown sklearn model: {name}")

In [ ]:
# 5. GPU-Accelerated Training Pipeline
import optuna
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.cuda.amp import autocast, GradScaler
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from src.data_loader import load_and_process_data, apply_smote
from src.models import SleepDataset, ANN, CNN, get_sklearn_model
import joblib
import os

DATA_PATH = "sleep_dataset.csv"
N_TRIALS = 20
EPOCHS = 20
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
USE_AMP = torch.cuda.is_available()
BATCH_SIZE = 512 if torch.cuda.is_available() else 64
SAVE_DIR = '/content/drive/MyDrive/Sleep_Disorder_Project'


def train_torch_model(model, train_loader, val_loader, epochs, lr):
    model.to(DEVICE)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    scaler = GradScaler(enabled=USE_AMP)
    best_val_f1 = 0.0

    for epoch in range(epochs):
        model.train()
        for X_batch, y_batch in train_loader:
            X_batch = X_batch.to(DEVICE, non_blocking=True)
            y_batch = y_batch.to(DEVICE, non_blocking=True)
            optimizer.zero_grad(set_to_none=True)
            with autocast(enabled=USE_AMP):
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

        model.eval()
        all_preds, all_labels = [], []
        with torch.no_grad(), autocast(enabled=USE_AMP):
            for X_batch, y_batch in val_loader:
                X_batch = X_batch.to(DEVICE, non_blocking=True)
                outputs = model(X_batch)
                _, preds = torch.max(outputs, 1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(y_batch.numpy())
        val_f1 = f1_score(all_labels, all_preds, average='weighted')
        if val_f1 > best_val_f1:
            best_val_f1 = val_f1
    return best_val_f1


def make_loaders(X_train, y_train, X_val, y_val):
    pin = torch.cuda.is_available()
    train_ds = SleepDataset(X_train, y_train)
    val_ds = SleepDataset(X_val, y_val)
    train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, pin_memory=pin, num_workers=2)
    val_dl = DataLoader(val_ds, batch_size=BATCH_SIZE, pin_memory=pin, num_workers=2)
    return train_dl, val_dl


def objective(trial, model_name, X_train_raw, X_val, y_train_raw, y_val, input_dim):
    X_train, y_train = apply_smote(X_train_raw, y_train_raw)

    if model_name == 'KNN':
        params = {
            'n_neighbors': trial.suggest_int('n_neighbors', 3, 15),
            'weights': trial.suggest_categorical('weights', ['uniform', 'distance']),
            'metric': trial.suggest_categorical('metric', ['euclidean', 'manhattan'])
        }
        model = get_sklearn_model('KNN', params)
        model.fit(X_train, y_train)
        return f1_score(y_val, model.predict(X_val), average='weighted')

    elif model_name == 'SVM':
        subsample_size = min(len(X_train), 20000)
        idx = np.random.choice(len(X_train), size=subsample_size, replace=False)
        X_sub, y_sub = X_train[idx], y_train[idx]
        params = {
            'C': trial.suggest_float('C', 0.1, 100.0, log=True),
            'kernel': trial.suggest_categorical('kernel', ['linear', 'rbf']),
            'gamma': trial.suggest_categorical('gamma', ['scale', 'auto'])
        }
        model = get_sklearn_model('SVM', params)
        model.fit(X_sub, y_sub)
        return f1_score(y_val, model.predict(X_val), average='weighted')

    elif model_name == 'RF':
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 100, 300),
            'max_depth': trial.suggest_int('max_depth', 10, 50),
            'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
            'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 4)
        }
        model = get_sklearn_model('RF', params)
        model.fit(X_train, y_train)
        return f1_score(y_val, model.predict(X_val), average='weighted')

    elif model_name in ['ANN', 'CNN']:
        lr = trial.suggest_float('lr', 1e-4, 1e-2, log=True)
        dropout = trial.suggest_float('dropout_rate', 0.1, 0.5)
        train_dl, val_dl = make_loaders(X_train, y_train, X_val, y_val)
        if model_name == 'ANN':
            model = ANN(input_dim,
                        trial.suggest_int('hidden_layers', 1, 3),
                        trial.suggest_int('units_per_layer', 32, 256),
                        dropout)
        else:
            model = CNN(input_dim,
                        trial.suggest_int('filters', 16, 64),
                        trial.suggest_int('kernel_size', 2, 3),
                        dropout)
        return train_torch_model(model, train_dl, val_dl, 10, lr)


def run_all():
    print(f"Device: {DEVICE}")
    if torch.cuda.is_available():
        print(f"GPU: {torch.cuda.get_device_name(0)}")
        print(f"Mixed Precision: Enabled | Batch Size: {BATCH_SIZE}")
    else:
        print("WARNING: No GPU. Training will be slower.")

    X_train_raw, X_test, y_train_raw, y_test, le = load_and_process_data(DATA_PATH)
    input_dim = X_train_raw.shape[1]
    print(f"Input dim: {input_dim}")

    X_opt_train, X_opt_val, y_opt_train, y_opt_val = train_test_split(
        X_train_raw, y_train_raw, test_size=0.2, random_state=42, stratify=y_train_raw
    )
    print(f"Opt split: train={X_opt_train.shape[0]}, val={X_opt_val.shape[0]}")

    models = ['KNN', 'SVM', 'RF', 'ANN', 'CNN']
    results = {}
    optuna.logging.set_verbosity(optuna.logging.WARNING)

    for m in models:
        print(f"\n{'='*50}")
        print(f"Optimizing {m}...")
        print(f"{'='*50}")

        study = optuna.create_study(direction='maximize')
        study.optimize(
            lambda t: objective(t, m, X_opt_train, X_opt_val, y_opt_train, y_opt_val, input_dim),
            n_trials=N_TRIALS
        )
        print(f"Best Params: {study.best_params}")
        print(f"Best Opt F1: {study.best_value:.4f}")

        print(f"Retraining final {m}...")
        X_train_full, y_train_full = apply_smote(X_train_raw, y_train_raw)

        if m in ['KNN', 'SVM', 'RF']:
            best_model = get_sklearn_model(m, study.best_params)
            best_model.fit(X_train_full, y_train_full)
            preds = best_model.predict(X_test)
            model_path = os.path.join(SAVE_DIR, f'{m}_best_model.joblib')
            joblib.dump(best_model, model_path)
            print(f"Saved {m} to {model_path}")
        else:
            p = study.best_params
            if m == 'ANN':
                best_model = ANN(input_dim, p['hidden_layers'], p['units_per_layer'], p['dropout_rate'])
            else:
                best_model = CNN(input_dim, p['filters'], p['kernel_size'], p['dropout_rate'])
            dl_train, dl_test = make_loaders(X_train_full, y_train_full, X_test, y_test)
            train_torch_model(best_model, dl_train, dl_test, epochs=EPOCHS, lr=p['lr'])
            model_path = os.path.join(SAVE_DIR, f'{m}_best_model.pth')
            torch.save(best_model.state_dict(), model_path)
            print(f"Saved {m} to {model_path}")
            best_model.eval()
            preds = []
            with torch.no_grad(), autocast(enabled=USE_AMP):
                for Xb, _ in dl_test:
                    out = best_model(Xb.to(DEVICE, non_blocking=True))
                    preds.extend(torch.max(out, 1)[1].cpu().numpy())

        acc = accuracy_score(y_test, preds)
        prec = precision_score(y_test, preds, average='weighted')
        rec = recall_score(y_test, preds, average='weighted')
        f1 = f1_score(y_test, preds, average='weighted')
        results[m] = {'Accuracy': acc, 'Precision': prec, 'Recall': rec, 'F1': f1}
        print(f"\n{m} Test -> Acc: {acc:.4f} | Prec: {prec:.4f} | Rec: {rec:.4f} | F1: {f1:.4f}")
        print(classification_report(y_test, preds, target_names=le.classes_))

    print("\n" + "="*60)
    print("FINAL RESULTS SUMMARY")
    print("="*60)
    print(pd.DataFrame(results).T.to_string())


run_all()